<div class="alert alert-danger">
    Read the following instructions carefully!
</div>

# Probability, Bayes' Theorem, (Conditional) Independence
## Problem Set 1
## Probabilistic Models UE

---
In the first assignment, you will familiarise yourself with matrix computations in NumPy. You must use operations on NumPy arrays, even if it would be possible to solve the exercises with simple multiplications, divisions, and loops. This will ensure that you get a feeling of how matrix operations and broadcasting works. If you are not familiar with these concepts, look at the interactive introduction to Python and the honey badger example.

**Hint:** You can still compute the correct results on paper and compare them with the solution produced by your Python code!


Before you start with this problem:
- Study the corresponding slide deck(s) and consider re-watching the lecture recording(s).
- Internalize the material until you feel confident you can work with them or implement them yourself. Only then start working on this problem; otherwise, you will waste a lot of time.

---


<div class="alert alert-warning">

**Due-Date:** see Moodle
   
**Constraints**: Operations on NumPy arrays only.
  
**Automatic Grading:** 

- Replace the placeholders `# YOUR CODE HERE` `raise NotImplementedError()` / `YOUR ANSWER HERE` with your code / answers.
- Put results in the corresponding variable; otherwise, we will not grade your solution (i.e., we assign 0 points).
- Do not delete or add cells.
    
**Submission:** As a ZIP-package via Moodle; the ZIP-package must have the following structure:
    
    + <student ID, (k/ vk + 8 digits), e.g. k01234567>.zip
    |
    |-- Problem_1.ipynb
    |-- ...
    |-- Problem_<# of problems>.ipynb
    +
    
**Questions?** Post it into the Problem Set Forum!
</div>



In [71]:
import numpy as np
from helpers import print_table

# 1. Inference-by-Enumeration (8 points)

The Inference-by-Enumeration algorithm computes the answer to a probabilistic query $P(\mathbf{X} \mid \mathbf{E=e})$ exactly from the full joint distribution table (FJDT).

---
### 1.1. Implementation


<div class="alert alert-warning">
Implement the Inference-by-Enumeration algorithm. (2 points)
</div>

Implement the `inference_by_enumeration` function for a generic probabilistic query of the form $P(\mathbf{X} \mid \mathbf{E})$. Note that this version of the Inference-by-Enumeration algorithm computes the probabilistic query for all possible assignments to the evidence variables, not only for one specific assignment (cf. slide deck: Probabilistic Models - Part 2: Fundamental Concepts and Notation, p. 40). The function must return one object:
- The answer to the probabilistic query, which is a `np.ndarray` with the same number of dimensions and the same variable order as the FJDT, but not the same size: The dimensions of non-query and non-evidence variables ($\mathbf{Z}$) must be converted to singleton dimensions, i.e., dimensions of size one.

For example, if we have a full joint distribution table of three binary variables (shape $2\times2\times2$) and we ask for the distributions of the first variable given the second variable, the result would be of shape $2\times2\times1$ (corresponding to two stacked conditional distribution tables).

**Hint:** Remember to solve this without a `for` loop. Set the `keepdims` parameter of NumPy's <a href="https://numpy.org/doc/stable/reference/generated/numpy.sum.html">sum</a> method to `True` to not discard the reduced dimensions. Keeping these empty dimensions simplifies <a href="https://numpy.org/doc/stable/user/basics.broadcasting.html">broadcasting operations</a> to a no-brainer.

In [133]:
def inference_by_enumeration(
    FJDT: np.ndarray, 
    query_variable_indices: tuple, 
    evidence_variable_indices: tuple = tuple()
) -> np.ndarray:
    '''
    Constructs a conditional probability table (CPT) from the full joint distribution.
    :param FJDT: The full joint distribution table as a np.ndarray.
    :param query_var: A tuple of indices representing the query variables.
    :param cond_var: A tuple of indices representing the conditioning variables.
    :returns: The conditional probability table (CPT) as a np.ndarray.
    '''
    assert type(FJDT) == np.ndarray, "FJDT must be a np.ndarray"
    assert type(query_variable_indices) == tuple, "query_variable_indices must be a tuple"
    assert type(evidence_variable_indices) == tuple, "evidence_variable_indices must be a tuple"
    
    # YOUR CODE HERE
   # print('FJDT:',FJDT)
   # print('query_variable_indices:',query_variable_indices)
    #print('evidence_variable_indices:',evidence_variable_indices)
    
    query_sum = query_variable_indices + evidence_variable_indices
    #print('query_sum:',query_sum)
    hv = tuple(set(range(FJDT.ndim)).difference(query_sum))
    #print('hv:',hv)
    p = np.sum(FJDT, axis=hv, keepdims=True)
    #print('p:',p)
    z = np.sum(p, axis=query_variable_indices, keepdims=True)
    #print('z:',z)
    CPT = 1/z * p
    #print(CPT)
   # print(type(CPT))
    return CPT

    raise NotImplementedError()

In [135]:
# create a full joint distribution table for three binary variables
ABC = np.ones((2,2,2)) / 2**3
# name the variable indices so we can refer to them more easily
A, B, C = 0, 1, 2

# check type & shape of result
assert type(inference_by_enumeration(ABC, (B, C), ())) == np.ndarray
# compute P(A)
assert inference_by_enumeration(ABC, (A,), ()).shape == (2, 1, 1)
# compute P(BC)
assert inference_by_enumeration(ABC, (B, C), ()).shape == (1, 2, 2)
# compute P(BC|A)
assert inference_by_enumeration(ABC, (B, C), (A,)).shape == (2, 2, 2)
# compute P(B|AC)
assert inference_by_enumeration(ABC, (B,), (C,A,)).shape == (2, 2, 2)


---
### 1.2.Computing Probabilities from a Full Joint Distribution Table

<br>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Atlantic_blue_marlin.jpg" width="500" height="600">
<br>

Based on his experience, Santiago, an old Cuban fisherman, has learned that temperature and precipitation are the most prominent factors influencing marlin fishing. After decades of (more or less) successful years, he decides to retire and pass on his knowledge as a full joint distribution table $P(C, R, H)$ on to you. You receive the following full joint distribution table:


<table style="border-collapse:collapse;border-spacing:0;width:500px"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center" rowspan="2">$P({C}, {R}, {H})$</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$\neg r$<br></th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$r$</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$\neg c$<br></td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.16</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.31</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.35</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.07<br></td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$c$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.09</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.01</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.004</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.006</td></tr></table>

In this table, $C$, $R$, and $H$ are the binary random variables encoding catch, rain, and hot, respectively. 
    
    
**Hint**: You can use `print_table` to print your probability distribution tables in a similar fashion.

In [138]:
help(print_table)

Help on function print_table in module helpers:

print_table(probability_table: numpy.ndarray, variable_names: str) -> None
    Prints a probability distribution table.

    Parameters
    ----------
    probability_table : np.ndarray
        The probability distribution table
    variable_names : str
        A string containing the variable names, e.g., 'CDE'.

    Returns
    -------
    None



<div class="alert alert-warning">
Create a NumPy array that contains the full joint distribution table $P(C, R, H)$ as defined above. <b>Important</b>: Encode $C$, $R$, and $H$ in the first, second, and third dimension of the NumPy array, respectively. Use index 0 for event *False* and index 1 for event *True*. (1 point)
</div>

In [141]:
CRH = None
# Check the result with print_table(CRH, 'CRH')

# remove the placeholder
# YOUR CODE HERE
CRH = np.array([[[0.16, 0.31], [0.35, 0.07]], [[0.09, 0.01], [0.004, 0.006]]])
C_, R_, H_ = 0, 1, 2
print_table(CRH,'crh')
#raise NotImplementedError()

,$r_0$,$r_0$,$r_1$,$r_1$
,$h_0$,$h_1$,$h_0$,$h_1$
$c_0$,0.160,0.310,0.350,0.070
$c_1$,0.090,0.010,0.004,0.006


In [143]:
assert CRH is not None, 'Store the result into the variable \'CRH\'!'
assert CRH.shape == (2,2,2), 'The full joint distribution table must have shape (2,2,2)'
assert np.isclose(CRH.sum(), 1, atol=1e-10), 'The probabilities of all atomic events must sum to one.'


---
### 1.3. Probabilistic Queries

Now check your implementation:
<div class="alert alert-warning">
Compute the probability distribution over catching a marlin given that the weather is <b>not</b> rainy. (3 points)
</div>

Compute this probabilistic query manually and then via `inference_by_enumeration`.

1. Manually: Keep your answer short and use $\LaTeX$ and Markdown. Write down
   - the *probabilistic query* (e.g., $P(X \mid Y=y)$) and
   - the *expression to compute the answer from the full joint distribution* (e.g., $P(X \mid Y=y) = \sum \dots$).

YOUR ANSWER HERE

P(C_0, R_0) = 0.16 + 0.31 = 0.47

P(C_1,RH_0) = 0.93 + 0.15 = 015

R(H_0) = 1625 + 01.3 + 0903 + 0105 = 578
5
P(C_R| H_0) = (P(CR0,H_0) )/RP(H_0) )  =47.0.57585 =824.
94
P(CR1| H_0) = (P(R_1,H_0) )R(P(H_0) )  1.0357.585 1750
.95RP(C|H_0) 824[,.91750.
095]R
P(C|H∑N▒N  ∑_R▒(C,〗     )/(∑ C,R(C,H_0) 〗)  


2. Think ybout the expected result: Give the *shape of the result* of the probabilistic query (without singleton dimensions) and the *number of non-redundant entries* in the result. Store your answer into the provided variables. Example:
 - the full joint distribution table of the previous example has a three dimensions with two entries each, thus it's shape is (2,2,2)
 - the full joint distribution table has $2*2*2$ entries; however one of them is redundant; thus the number of non-redundant entries is $2*2*2 - 1$.

In [150]:
probability_table_shape = None # e.g., (2,2,2) for the FJDT, (2,) for a vector, () for a scalar
number_non_redundant_elements = None # e.g., 2*2*2 - 1 for the FJDT

# remove the placeholder
# YOUR CODE HERE
probability_table_shape = (2,)
number_non_redundant_elements = 2

#raise NotImplementedError()

In [152]:
assert type(probability_table_shape) is tuple, 'Shape of the result must be a tuple.'

In [154]:
assert type(number_non_redundant_elements) is int, 'Number of elements must be int.'

3. Check your answer with the `inference_by_enumeration` method and store the result into the provided variable. **If necessary, select the result for the given evidence and remove all singleton dimensions.**

In [157]:
C_not_r = None # Use inference_by_enumeration to compute the result. Select the result for the given evidence (if any) and discard singleton dimensions.
# The implementation of algorithm
c_not_r = inference_by_enumeration(CRH, (C_,) ,(not R_,1 ))
C_not_r = c_not_r[:, 0] 
C_not_r = np.squeeze(C_not_r)
print("Result:", C_not_r)

Result: [0.8245614 0.1754386]


In [159]:
C_not_r = None # Use inference_by_enumeration to compute the result. Select the result for the given evidence (if any) and discard singleton dimensions.
# YOUR CODE HERE
c = inference_by_enumeration(CRH, (C_,), (R_,))
c = c[:, 0] 
C_not_r = c.squeeze()

#raise NotImplementedError()
print("Result:")
print_table(C_not_r,"c")

Result:


$c_0$,0.825
$c_1$,0.175


In [161]:
assert C_not_r is not None, 'Store the result into the variable \'C_not_r\'!'


---
### 1.4. Independence



<div class="alert alert-warning">
Implement an algorithm which checks if two variables are independent based on a FJDT. (2 points)
</div>

Implement the `check_independence` method which returns `True` if two variables A and B are independent of each other and `False` otherwise, given a FJDT.

**Hint:** use the `inference_by_enumeration` to compute the joint distribution and the marginal distributions. The product of the marginals is simply a multiplication due to the aligned singelton dimensions. Use `np.allclose` to avoid numerical issues when comparing the distributions.




In [164]:
def check_independence(FJDT: np.ndarray, A: int, B: int):
    '''
    Constructs a conditional probability table (CPT) from the full joint distribution.
    :param FJDT: The full joint distribution table as a np.ndarray.
    :param A: Index of variable A.
    :param B: Index of variable B.
    :returns: True if A is independent of B and False otherwise.
    '''
    assert type(FJDT) == np.ndarray, "FJDT must be a np.ndarray"
    assert type(A) == int, "A must be a int"
    assert type(B) == int, "B must be a int"
    assert 0 <= A < FJDT.ndim, "invalid variable index for A"
    assert 0 <= B < FJDT.ndim, "invalid variable index for B"
    
    # YOUR CODE HERE
    P_A = inference_by_enumeration(FJDT, (A,), ())
    P_B = inference_by_enumeration(FJDT, (B,), ())
    P_AB = inference_by_enumeration(FJDT, (A,B), ())
    print('A:',P_A)
    print('B:',P_B)
    print('AB:',P_AB)
    return np.allclose(P_AB, P_A*P_B)

    #raise NotImplementedError()

    #return False

In [166]:
assert check_independence(np.ones((2,3))/ (2*3), 0, 1) in [True, False], 'Results must be a boolean'

A: [[0.5]
 [0.5]]
B: [[0.33333333 0.33333333 0.33333333]]
AB: [[0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667]]


Let's check if Catch is independent of Rain:

In [169]:
check_independence(CRH, 0, 1)

A: [[[0.89]]

 [[0.11]]]
B: [[[0.57]
  [0.43]]]
AB: [[[0.47]
  [0.42]]

 [[0.1 ]
  [0.01]]]


False